# Topic Modeling (LDA)
## Setup


In [1]:
!pip install --upgrade gensim

In [2]:
!pip install nltk
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00


## import libraries

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import numpy as np
import json
import glob

#gensim
import gensim
import gensim.corpora as copora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data Prep

In [5]:
def laod_data(file):
  with open (file, "r", encoding="utf-8") as f:
    data = json.load(f)
  return (data)

def write_data(file, data):
  with open (file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

In [6]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [31]:
data = laod_data("/content/Extraversion_answers_and_comments.json")["comments"]

print(data[0][0:90])

@baptiste : The same minus the error. All techniques I explained are used in package help 


In [32]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return(texts_out)

lemmatized_text = lemmatization(data)
print(lemmatized_text[0][0:90])

same error technique explain use package help page tutorial presentation give r


In [33]:
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_words = gen_words(lemmatized_text)

print(data_words[0][0:30])

['same', 'error', 'technique', 'explain', 'use', 'package', 'help', 'page', 'tutorial', 'presentation', 'give']


In [34]:
id2word = gensim.corpora.Dictionary(data_words)

corpus = []

for text in data_words:
  new = id2word.doc2bow(text) # bow -> bag of words
  corpus.append(new)

print(corpus[0][0:20]) #each tuple contains index of the word along with the frequency of that word.

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]


In [35]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=10,
                                    random_state=100,
                                    update_every=1,
                                    chunksize=100,
                                    passes=10,
                                    alpha="auto")

## Visualization of Data

In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word, mds="mmds", R=9)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.359241  0.261003       1        1  31.238238
3      0.006337 -0.409900       2        1  14.158965
2     -0.374466 -0.186392       3        1  14.116300
8     -0.027362  0.372092       4        1   8.576118
5      0.280568 -0.219059       5        1   7.766046
7      0.393960  0.058665       6        1   6.716911
4      0.195399  0.253835       7        1   6.237189
1     -0.036362 -0.170863       8        1   4.422246
0     -0.167074  0.020067       9        1   4.250615
9      0.088241  0.020553      10        1   2.517372, topic_info=            Term        Freq       Total Category  logprob  loglift
186         just  267.000000  267.000000  Default   9.0000   9.0000
156         file  130.000000  130.000000  Default   8.0000   8.0000
44        answer  325.000000  325.000000  Default   7.0000   7.0000
10           use  467.000000  467.000000  Default   6.0000   6.0000
306         good  130.000000  130.000000  Default   5.0000   5.0000
...          ...         ...         ...      ...      ...      ...
2041        font   12.633056   13.482574  Topic10  -3.9024   3.6169
304       detail   10.799519   11.649065  Topic10  -4.0592   3.6062
1087   basically   10.246161   11.095799  Topic10  -4.1118   3.6023
165   comparison    9.532498   10.382159  Topic10  -4.1840   3.5966
1925     compare    8.244563    9.094113  Topic10  -4.3291   3.5839

[111 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2901      6  0.991079  abstraction
1268      7  0.047009       access
1268      8  0.940176       access
19        1  0.087639          add
19        2  0.876392          add
...     ...       ...          ...
124       3  0.986757        value
96        1  0.642322         want
96        3  0.353526         want
59        2  0.993567          way
43        1  0.995025         work

[112 rows x 3 columns], R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 3, 9, 6, 8, 5, 2, 1, 10])

In [37]:
print('\nPerplexity: ', lda.log_perplexity(corpus,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.964112001061001

Coherence Score:  0.41825401490847314
